# **Install Google Ads Module**

In [1]:
pip install google-ads==21.3.0

# **Input Google Ads yaml**

In [2]:
from google.ads.googleads.client import GoogleAdsClient
import yaml
from datetime import date
today_date = date.today()
today_date_str = today_date.strftime('%Y-%m-%d')

client = GoogleAdsClient.load_from_storage("/google-ads.yaml")

# Load Google Ads API configuration from the YAML file
with open("/google-ads.yaml", "r") as file:
    config = yaml.safe_load(file)

# Access the login_customer_ids list from the loaded configuration
login_customer_ids = config.get("login_customer_ids", [])

# Use login_customer_ids as needed in your code
for login_customer_id in login_customer_ids:
    print("Login Customer ID:", login_customer_id)

Login Customer ID: 1806447736
Login Customer ID: 5638486708
Login Customer ID: 3754127381
Login Customer ID: 1932652414
Login Customer ID: 4765621837


# **Request data of Pmax Campaign(From Campaign)**

In [3]:
import pandas as pd
def main(client, customer_id):
    ga_service = client.get_service("GoogleAdsService")

    query = f"""
        SELECT
            customer.id,
            customer.descriptive_name,
            segments.year,
            segments.quarter,
            segments.date,
            campaign.id,
            campaign.name,
            campaign.advertising_channel_type,
            segments.device,
            metrics.cost_micros,
            metrics.clicks,
            metrics.impressions,
            metrics.engagements,
            metrics.video_views
        FROM campaign
        where segments.date BETWEEN '2021-01-14' AND '{today_date_str}'
        ORDER BY customer.id DESC"""
    stream = ga_service.search_stream(customer_id=customer_id, query=query)

    rows = []

    for batch in stream:
        for row in batch.results:
            final_url = row.ad_group_ad.ad.final_urls[0] if row.ad_group_ad.ad.final_urls else ''
            rows.append({
                'Account_Id': str(row.customer.id),
                'Account_Name': str(row.customer.descriptive_name),
                'Date': str(row.segments.date),
                'Year': float(row.segments.year),
                'Quarter': str(row.segments.quarter),
                'Campaign_Id': str(row.campaign.id),
                'Campaign_Name': str(row.campaign.name),
                'Campaign_Type': str(row.campaign.advertising_channel_type),
                'Adgroup_Id': str(row.ad_group.id if row.ad_group.id else ''),
                'Ad_Group': str(row.ad_group.name if row.ad_group.name else ''),
                'Ad_Id': str(row.ad_group_ad.ad.id if row.ad_group_ad.ad.id else ''),
                'Ad': str(row.ad_group_ad.ad.name if row.ad_group_ad.ad.name else ''),
                'Final_URLs': str(final_url),
                'Device': str(row.segments.device if row.segments.device else ''),
                'Network_Type': str(row.segments.ad_network_type),
                'Clicks': float(row.metrics.clicks),
                'Impressions': float(row.metrics.impressions),
                'Engagements': float(row.metrics.engagements),
                'Video_Views': float(row.metrics.video_views),
                'Cost_Micros':float(row.metrics.cost_micros)
            })
    df = pd.DataFrame(rows)
    return (df)
df = pd.concat([main(client, '5638486708'),main(client, '1806447736'),main(client, '3754127381'),main(client, '1932652414'),main(client, '4765621837')])

# **Request data of Non-Pmax Campaign(From Ad Group Ad)**

In [4]:
def main(client, customer_id):
    ga_service = client.get_service("GoogleAdsService")

    query = f"""
        SELECT
            customer.id,
            customer.descriptive_name,
            segments.year,
            segments.quarter,
            segments.date,
            campaign.id,
            campaign.name,
            campaign.advertising_channel_type,
            ad_group.id,
            ad_group.name,
            ad_group_ad.ad.id,
            ad_group_ad.ad.name,
            ad_group_ad.ad.responsive_display_ad.headlines,
            ad_group_ad.ad.responsive_search_ad.headlines,
            ad_group_ad.ad.final_urls,
            segments.ad_network_type,
            metrics.cost_micros,
            metrics.clicks,
            metrics.impressions,
            metrics.engagements,
            metrics.video_views
        FROM ad_group_ad
        where segments.date BETWEEN '2021-01-14' AND '{today_date_str}'
        ORDER BY ad_group_ad.ad.id DESC"""
    stream = ga_service.search_stream(customer_id=customer_id, query=query)

    rows = []

    for batch in stream:
        for row in batch.results:
            final_url = row.ad_group_ad.ad.final_urls[0] if row.ad_group_ad.ad.final_urls else ''
            rows.append({
                'Account_Id': str(row.customer.id),
                'Account_Name': str(row.customer.descriptive_name),
                'Date': str(row.segments.date),
                'Year': float(row.segments.year),
                'Quarter': str(row.segments.quarter),
                'Campaign_Id': str(row.campaign.id),
                'Campaign_Name': str(row.campaign.name),
                'Campaign_Type': str(row.campaign.advertising_channel_type),
                'Adgroup_Id': str(row.ad_group.id if row.ad_group.id else ''),
                'Ad_Group': str(row.ad_group.name if row.ad_group.name else ''),
                'Ad_Id': str(row.ad_group_ad.ad.id if row.ad_group_ad.ad.id else ''),
                'Ad': str(row.ad_group_ad.ad.name if row.ad_group_ad.ad.name else ''),
                'Display_Title': str(row.ad_group_ad.ad.responsive_display_ad.headlines[0].text if row.ad_group_ad.ad.responsive_display_ad.headlines else ''),
                'Search_Title': str(row.ad_group_ad.ad.responsive_search_ad.headlines[0].text if row.ad_group_ad.ad.responsive_search_ad.headlines else''),
                'Final_URLs': str(final_url),
                'Device': str(row.segments.device if row.segments.device else ''),
                'Network_Type': str(row.segments.ad_network_type),
                'Clicks': float(row.metrics.clicks),
                'Impressions': float(row.metrics.impressions),
                'Engagements': float(row.metrics.engagements),
                'Video_Views': float(row.metrics.video_views),
                'Cost_Micros':float(row.metrics.cost_micros)
            })

    df2 = pd.DataFrame(rows)
    return (df2)
df2 = pd.concat([main(client, '5638486708'),main(client, '1806447736'),main(client, '3754127381'),main(client, '1932652414'),main(client, '4765621837')])

# **Organize data of Non-Pmax Campaign structure**

In [5]:
df2.reset_index(drop=True, inplace=True)
df2.loc[(df2['Ad'] == '') & (df2['Campaign_Type'] == 'AdvertisingChannelType.DISPLAY'), 'Ad'] = df2['Display_Title']
df2.loc[(df2['Ad'] == '') & (df2['Campaign_Type'] == 'AdvertisingChannelType.SEARCH'), 'Ad'] = df2['Search_Title']
df2.drop(columns=['Search_Title', 'Display_Title'], inplace=True)
df2.loc[(df2['Campaign_Type'] == 'AdvertisingChannelType.SHOPPING') | (df2['Ad'] == 'AdvertisingChannelType.DISCOVERY'), 'Ad'] = ''

# **Connect Pmax and Non-Pmax data**

In [6]:
df_pmax = df[(df['Campaign_Type'] == 'AdvertisingChannelType.PERFORMANCE_MAX') | (df['Campaign_Type'] == 'AdvertisingChannelType.LOCAL')]
df_wo_pmax = df2[(df2['Campaign_Type'] != 'AdvertisingChannelType.PERFORMANCE_MAX') & (df2['Campaign_Type'] != 'AdvertisingChannelType.LOCAL')]
df_all = pd.concat([df_wo_pmax, df_pmax], ignore_index=True)

# **Obtain exchange rate data for each region from GCP**

In [8]:
import numpy as np
from datetime import datetime, date
import os
from google.cloud import bigquery as bq

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/asus-data-and-insights-team-55deef837577 1.json'
client = bq.Client()

query_cur = '''
SELECT *
FROM `asus-data-and-insights-team.Temp_Media_Data.Currency_List_Mapping`
'''
cur = client.query(query_cur).result().to_arrow(create_bqstorage_client=True).to_pandas()

query_am = '''
SELECT *
FROM `asus-data-and-insights-team.Temp_Media_Data.Media_Account_Mapping`
'''
am = client.query(query_am).result().to_arrow(create_bqstorage_client=True).to_pandas()

df_am_cur = pd.merge(am, cur[['year', 'quarter', 'date_quarter','payment_profile','payment_region', 'From_Currency', 'To_Currency', 'exchange_rate']], left_on = 'payment_region', right_on = 'payment_region', how = 'inner')
df_am_cur['date_quarter'] = df_am_cur['date_quarter'].astype(str)

# **Merge all Tables and perform data conversion on some variables**

In [9]:
df_am_cur_all = pd.merge(df_all, df_am_cur[['account_id', 'provider', 'product_bg', 'product_line','exchange_rate','year','date_quarter','account_region','account_country']], left_on=['Account_Id','Year','Quarter'], right_on=['account_id','year','date_quarter'], how='inner')
df_am_cur_all['Campaign_Type'].replace({'AdvertisingChannelType.PERFORMANCE_MAX': 'Pmax', 'AdvertisingChannelType.LOCAL': 'Pmax', 'AdvertisingChannelType.SEARCH': 'Search', 'AdvertisingChannelType.VIDEO': 'Video', 'AdvertisingChannelType.DISPLAY': 'Display', 'AdvertisingChannelType.SHOPPING': 'Shopping', 'AdvertisingChannelType.DISCOVERY': 'Discovery & Demand gen', 'AdvertisingChannelType.DEMAND GEN': 'Discovery & Demand gen', 'AdvertisingChannelType.DEMAND_GEN': 'Discovery & Demand gen'}, inplace=True)
df_am_cur_all['Funnel_Strategy'] = np.where(df_am_cur_all['Campaign_Type'] == 'Pmax', 'Conversion',
                                            np.where(df_am_cur_all['Campaign_Type'] == 'Video', 'Awareness',
                                                     np.where(df_am_cur_all['Campaign_Type'] == 'Display', 'Consideration',
                                                              np.where(df_am_cur_all['Campaign_Type'] == 'Discovery & Demand gen', 'Consideration',
                                                                      np.where(df_am_cur_all['Campaign_Type'] == 'Search', 'Conversion', 'Other')))))
df_am_cur_all['Cost_USD'] = df_am_cur_all['Cost_Micros'] / 1000000 * df_am_cur_all['exchange_rate']
df_am_cur_all = df_am_cur_all.rename(columns={'exchange_rate': 'Exchange_Rate', 'provider': 'Platform','account_region':'Account_Region','account_country':'Account_Country','product_bg':'Product_BG', 'product_line':'Product_Line'})
df_gcp = df_am_cur_all[['Date','Exchange_Rate','Platform','Account_Id','Account_Name','Account_Region','Account_Country','Campaign_Name','Campaign_Id','Ad_Group','Adgroup_Id','Ad','Ad_Id','Impressions','Clicks','Video_Views','Engagements','Cost_USD','Campaign_Type','Funnel_Strategy','Final_URLs','Product_BG','Product_Line','Device','Network_Type']]
df_gcp['Device'] = df_gcp['Device'].str.replace('Device.', '')
df_gcp['Network_Type'] = df_gcp['Network_Type'].str.replace('AdNetworkType.', '')

<ipython-input-9-ec208aa5a2c2>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gcp['Device'] = df_gcp['Device'].str.replace('Device.', '')
<ipython-input-9-ec208aa5a2c2>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gcp['Network_Type'] = df_gcp['Network_Type'].str.replace('AdNetworkType.', '')


# **Define the Type of each field data**

In [10]:
df_gcp['Date'] = pd.to_datetime(df_gcp['Date'])
df_gcp['Exchange_Rate'] = df_gcp['Exchange_Rate'].astype(float)
df_gcp['Impressions'] = df_gcp['Impressions'].astype(float)
df_gcp['Clicks'] = df_gcp['Clicks'].astype(float)
df_gcp['Video_Views'] = df_gcp['Video_Views'].astype(float)
df_gcp['Engagements'] = df_gcp['Engagements'].astype(float)
df_gcp['Cost_USD'] = df_gcp['Cost_USD'].astype(float)
df_gcp['Platform'] = df_gcp['Platform'].astype(str)
df_gcp['Account_Id'] = df_gcp['Account_Id'].astype(str)
df_gcp['Account_Name'] = df_gcp['Account_Name'].astype(str)
df_gcp['Account_Region'] = df_gcp['Account_Region'].astype(str)
df_gcp['Account_Country'] = df_gcp['Account_Country'].astype(str)
df_gcp['Campaign_Name'] = df_gcp['Campaign_Name'].astype(str)
df_gcp['Campaign_Id'] = df_gcp['Campaign_Id'].astype(str)
df_gcp['Ad_Group'] = df_gcp['Ad_Group'].astype(str)
df_gcp['Adgroup_Id'] = df_gcp['Adgroup_Id'].astype(str)
df_gcp['Ad'] = df_gcp['Ad'].astype(str)
df_gcp['Ad_Id'] = df_gcp['Ad_Id'].astype(str)
df_gcp['Campaign_Type'] = df_gcp['Campaign_Type'].astype(str)
df_gcp['Funnel_Strategy'] = df_gcp['Funnel_Strategy'].astype(str)
df_gcp['Final_URLs'] = df_gcp['Final_URLs'].astype(str)
df_gcp['Product_BG'] = df_gcp['Product_BG'].astype(str)
df_gcp['Product_Line'] = df_gcp['Product_Line'].astype(str)
df_gcp['Device'] = df_gcp['Device'].astype(str)
df_gcp['Network_Type'] = df_gcp['Network_Type'].astype(str)

<ipython-input-10-353554fdf25d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gcp['Date'] = pd.to_datetime(df_gcp['Date'])
<ipython-input-10-353554fdf25d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gcp['Exchange_Rate'] = df_gcp['Exchange_Rate'].astype(float)
<ipython-input-10-353554fdf25d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

# **Create a Table in GCP**

In [ ]:
table_name = 'asus-data-and-insights-team.Jd_Wu_US.googleapi_demo_0528'

In [ ]:
#IF table is new
table = bq.Table(table_name)
table.schema = (
        bq.SchemaField('Date',      'DATE'),
        bq.SchemaField('Exchange_Rate',      'FLOAT'),
        bq.SchemaField('Platform',      'STRING'),
        bq.SchemaField('Account_Id',     'STRING'),
        bq.SchemaField('Account_Name',   'STRING'),
        bq.SchemaField('Account_Region',  'STRING' ),
        bq.SchemaField('Account_Country', 'STRING' ),
        bq.SchemaField('Campaign_Name', 'STRING' ),
        bq.SchemaField('Campaign_Id', 'STRING' ),
        bq.SchemaField('Ad_Group', 'STRING' ),
        bq.SchemaField('Adgroup_Id', 'STRING' ),
        bq.SchemaField('Ad', 'STRING' ),
        bq.SchemaField('Ad_Id', 'STRING' ),
        bq.SchemaField('Impressions', 'FLOAT' ),
        bq.SchemaField('Clicks', 'FLOAT' ),
        bq.SchemaField('Video_Views', 'FLOAT' ),
        bq.SchemaField('Engagements', 'FLOAT' ),
        bq.SchemaField('Cost_USD', 'FLOAT' ),
        bq.SchemaField('Campaign_Type', 'STRING' ),
        bq.SchemaField('Funnel_Strategy', 'STRING' ),
        bq.SchemaField('Final_URLs', 'STRING' ),
        bq.SchemaField('Product_BG', 'STRING' ),
        bq.SchemaField('Product_Line', 'STRING' ),
        bq.SchemaField('Device', 'STRING' ),
        bq.SchemaField('Network_Type', 'STRING' ),
)
client.create_table(table)

Table(TableReference(DatasetReference('asus-data-and-insights-team', 'Jd_Wu_US'), 'googleapi_demo_0528'))

In [ ]:
#IF table already exist
table_ref = table_name
if client.get_table(table_ref):
    client.delete_table(table_ref)

table = bq.Table(table_name)
table.schema = (
        bq.SchemaField('Date',      'DATE'),
        bq.SchemaField('Exchange_Rate',      'FLOAT'),
        bq.SchemaField('Platform',      'STRING'),
        bq.SchemaField('Account_Id',     'STRING'),
        bq.SchemaField('Account_Name',   'STRING'),
        bq.SchemaField('Account_Region',  'STRING' ),
        bq.SchemaField('Account_Country', 'STRING' ),
        bq.SchemaField('Campaign_Name', 'STRING' ),
        bq.SchemaField('Campaign_Id', 'STRING' ),
        bq.SchemaField('Ad_Group', 'STRING' ),
        bq.SchemaField('Adgroup_Id', 'STRING' ),
        bq.SchemaField('Ad', 'STRING' ),
        bq.SchemaField('Ad_Id', 'STRING' ),
        bq.SchemaField('Impressions', 'FLOAT' ),
        bq.SchemaField('Clicks', 'FLOAT' ),
        bq.SchemaField('Video_Views', 'FLOAT' ),
        bq.SchemaField('Engagements', 'FLOAT' ),
        bq.SchemaField('Cost_USD', 'FLOAT' ),
        bq.SchemaField('Campaign_Type', 'STRING' ),
        bq.SchemaField('Funnel_Strategy', 'STRING' ),
        bq.SchemaField('Final_URLs', 'STRING' ),
        bq.SchemaField('Product_BG', 'STRING' ),
        bq.SchemaField('Product_Line', 'STRING' ),
        bq.SchemaField('Device', 'STRING' ),
        bq.SchemaField('Network_Type', 'STRING' ),
)
client.create_table(table)

In [ ]:
import time

print('Loading data into ' + table_name)
load_job = client.load_table_from_dataframe(df_gcp, table)

while load_job.running():
  print('waiting for the load job to complete')
  time.sleep(1)

if load_job.errors == None:
  print('Load complete!')
else:
  print(load_job.errors)

Loading data into asus-data-and-insights-team.Jd_Wu_US.googleapi_demo_0528
waiting for the load job to complete
waiting for the load job to complete
waiting for the load job to complete
Load complete!
